In [1]:
import copy
import pickle
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.cm import get_cmap
from formation_map import get_formation
from genetic_algorithm import initialise_individual, select_individuals, mutate_individual, crossover_individuals
from plotting import animate
from speciation import speciate, penalise_overpopulation, get_genetic_diversity

In [2]:
cmap = get_cmap("viridis")

In [3]:
# Matplotlib default changes.
mpl.rcParams["figure.facecolor"] = "black"
mpl.rcParams["figure.figsize"] = (12, 10)
mpl.rcParams["axes.facecolor"] = "black"
mpl.rcParams["axes.edgecolor"] = "white"
mpl.rcParams["axes.labelcolor"] = "white"
mpl.rcParams["font.size"] = 14
mpl.rcParams["grid.color"] = "white"
mpl.rcParams["savefig.facecolor"] = "black"
mpl.rcParams["text.color"] = "white"
mpl.rcParams["xtick.color"] = "white"
mpl.rcParams["ytick.color"] = "white"
mpl.rcParams["hatch.color"] = "white"
mpl.rcParams["lines.color"] = "white"
mpl.rcParams["legend.edgecolor"] = "none"
mpl.rc('animation', html='html5')

In [4]:
np.random.seed(42)

In [35]:
N_SAMPLES = 10000

N_INDIVIDUALS = 100
N_GENERATIONS = 1000

REPLACE_MUTATION_PROBABILITY = 0.75
SWAP_MUTATION_PROBABILITY = 0.2
SWITCH_MUTATION_PROBABILITY = 0.05
CROSSOVER_PROBABILITY = 0.05

RATING_WEIGHT = 10
CHEMISTRY_WEIGHT = 10
PRICE_WEIGHT = 0.1
N_BARCELONA_PLAYERS_WEIGHT = 1000

SPECIATION = False
SPECIATION_THRESHOLD = 2.5

In [36]:
formation = get_formation("433")

In [37]:
def loss(formation):
    rating = min(85 * 11, formation.get_rating()) * RATING_WEIGHT
    chemistry = min(80, formation.get_chemistry()) * CHEMISTRY_WEIGHT
    price = -formation.get_price() * PRICE_WEIGHT
    n_barcelona_players = min(1, len([p for p in formation.slots.values() if p.player.data["club"] == "FC Barcelona"])) * N_BARCELONA_PLAYERS_WEIGHT
    return rating + chemistry + price + n_barcelona_players

In [38]:
fitness_log = []
rating_log = []
chemistry_log = []
price_log = []
champion_log = []
diversity_log = []

champion = None
population = [initialise_individual(formation) for _ in range(N_INDIVIDUALS)]
for individual in population:
    individual.evaluate(loss)


for i in range(N_GENERATIONS):
    if SPECIATION:
        species = speciate(population, SPECIATION_THRESHOLD)
        diversity_log.append(get_genetic_diversity(species))
        penalise_overpopulation(species)
    population = select_individuals(population)
    for j in range(int(len(population)/2)):
        individuals = population[2*j:2*j+2]
        if np.random.uniform() < CROSSOVER_PROBABILITY:
            crossover_individuals(*individuals, formation)
        for individual in individuals:
            mutate_individual(individual, REPLACE_MUTATION_PROBABILITY, SWAP_MUTATION_PROBABILITY, SWITCH_MUTATION_PROBABILITY)
            individual.evaluate(loss)
    challenger = population[np.argmax(population)]
    new_champion = False
    if not champion or challenger > champion:
        champion = copy.copy(challenger)
        champion_log.append(champion)
        new_champion = True
    champion.fill()
    fitness_log.append(champion.fitness)
    rating_log.append(champion.formation.get_rating())
    chemistry_log.append(champion.formation.get_chemistry())
    price_log.append(champion.formation.get_price())
    if new_champion:   
        print("TURN {}".format(i))
        print("PLAYERS:")
        for k, v in champion.player_map.items():
            print("\t{}: {}".format(k, v))
        print("FITNESS: {0:.2f}".format(champion.fitness))
        print("RATING: {}".format(champion.formation.get_rating()))
        print("CHEMISTRY: {}".format(champion.formation.get_chemistry()))
        print("PRICE: {}".format(int(champion.formation.get_price())))
        print("")
        
print("FINAL CHAMPION")
print("PLAYERS:")
for k, v in champion.player_map.items():
    print("\t{}: {}".format(k, v))
print("FITNESS: {0:.2f}".format(champion.fitness))
print("RATING: {}".format(champion.formation.get_rating()))
print("CHEMISTRY: {}".format(champion.formation.get_chemistry()))
print("PRICE: {}".format(int(champion.formation.get_price())))
print("")

TURN 0
PLAYERS:
	GK: Player(id=15364, name=Hawsawi, overall=55, position=GK, price=200, nationality=Saudi Arabia, league=Saudi Professional League, club=Al Fayha)
	LB: Player(id=10577, name=Pogatetz, overall=68, position=CB, price=500, nationality=Austria, league=Österreichische Fußball-Bundesliga, club=LASK Linz)
	CB1: Player(id=13640, name=Brown, overall=65, position=LB, price=200, nationality=England, league=EFL League One, club=Coventry City)
	CB2: Player(id=3028, name=Callsen-Bracker, overall=71, position=CB, price=600, nationality=Germany, league=Bundesliga, club=FC Augsburg)
	RB: Player(id=6996, name=Carraça, overall=71, position=RB, price=800, nationality=Portugal, league=Liga NOS, club=Boavista FC)
	CM1: Player(id=12916, name=Hernanes, overall=78, position=CM, price=1200, nationality=Brazil, league=CSL, club=Hebei China Fortune FC)
	CM2: Player(id=12805, name=Fernández, overall=80, position=CM, price=800, nationality=Argentina, league=CSL, club=Beijing Renhe)
	CM3: Player(id=9

In [ ]:
with open("../input/champion_log.p", "wb") as f:
    pickle.dump(champion_log, f)

In [ ]:
x = np.arange(0, N_GENERATIONS)
fig, axs = plt.subplots(4, 1, figsize=(16, 12))

for ax, data, title in zip(axs, [fitness_log, rating_log, chemistry_log, price_log], ["Fitness", "Rating", "Chemistry", "Price"]):
    ax.plot(x, data, linewidth=3, color=cmap(1.))
    ax.set_title(label=title)
    ax.set_xlabel("Generation")    
    
fig.tight_layout()

plt.savefig("../images/fitness_curve.png", dpi=120)